***CHI-SQUARE FOR FEATURE SLEECTION***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

import yfinance as yf

In [2]:
stock = 'AAPL'
start = '2004-01-01'
end = '2024-01-01'

dataset = yf.download(stock, start, end)
dataset.columns = dataset.columns.get_level_values(0)
dataset.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2004-01-02,0.319777,0.326839,0.318274,0.323833,144642400
2004-01-05,0.333151,0.336456,0.321880,0.321880,395018400
2004-01-06,0.331948,0.336907,0.326239,0.334352,509348000
2004-01-07,0.339462,0.343069,0.329544,0.332099,586874400
2004-01-08,0.351033,0.356593,0.340363,0.343219,460303200


In [5]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Close'].shift(-1) > dataset['Close'],1,0)
dataset['Returns'] = dataset['Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

Price,Close,High,Low,Open,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,
2004-01-06,0.331948,0.336907,0.326239,0.334352,509348000,1,0,1,-0.003610
2004-01-07,0.339462,0.343069,0.329544,0.332099,586874400,0,1,1,0.022636
2004-01-08,0.351033,0.356593,0.340363,0.343219,460303200,0,1,0,0.034085
2004-01-09,0.345623,0.362604,0.342467,0.349079,427459200,1,1,1,-0.015412
2004-01-12,0.356593,0.360650,0.347126,0.349380,487547200,1,1,1,0.031740


In [6]:
dataset.shape

(5031, 9)

In [7]:
X = dataset[['Open', 'High', 'Low','Close', 'Volume', 'Buy_Sell_on_Open', 'Buy_Sell', 'Returns']]
X = X.astype(int)
y = dataset['Increase_Decrease']

In [8]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [10]:
#select two features with highest chi-squared statistics

chi2_selector = SelectKBest(chi2, k = 2)
x_kbest = chi2_selector.fit_transform(X, y)

print('Original number of features:', X.shape[1])
print('Reduced number of features:', x_kbest.shape[1])

Original number of features: 8
Reduced number of features: 2


In [11]:
chi2_selector.scores_

array([1.43028247e+00, 3.06438728e+00, 4.98127993e-01, 1.70776324e+00,
       3.38435213e+10, 1.65596658e+00, 3.88449375e+00,            nan])

In [12]:
chi2_selector.pvalues_

array([0.23171834, 0.08002533, 0.48032382, 0.19127584, 0.        ,
       0.19814879, 0.04873395,        nan])

In [13]:
names = X.columns.values[chi2_selector.get_support()]
scores = chi2_selector.scores_[chi2_selector.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'Chi-Squared'])
#Sort the dataframe for better visualization
ns_df_sorted = ns_df.sort_values(['Chi-Squared', 'Feat_names'], ascending = [False, True])
print(ns_df_sorted)

  Feat_names   Chi-Squared
0     Volume  3.384352e+10
1   Buy_Sell  3.884494e+00
